In [1]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')
from ml_pipeline import train_n_predict, validation, clean_split
from Data.data_cleaner import cleaner

# Clean Data

In [2]:
## Clean data
train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/imputed_morph_embed.csv"
X_train, X_val, X_query, y_train, y_val, y_query = clean_split(train_path, feature_path, morph_path)

In [3]:
# Oversample X_train_feat
ros = RandomOverSampler(random_state=0, sampling_strategy = 'minority')
X_train, y_train = ros.fit_resample(
        X_train, y_train)

In [4]:
X_train_feat = X_train.copy()
X_val_feat = X_val.copy()
X_query_feat = X_query.copy()

# Identify Most Important FW

In [82]:
X_train_fw = X_train.copy() # make a copy of the original X_train
fw = pd.read_csv(feature_path)
X_train_fw

,nucleus_id,feature_weight_0,feature_weight_1,feature_weight_2,feature_weight_3,feature_weight_4,feature_weight_5,feature_weight_6,feature_weight_7,feature_weight_8,...,feature_weight_502,feature_weight_503,feature_weight_504,feature_weight_505,feature_weight_506,feature_weight_507,feature_weight_508,feature_weight_509,feature_weight_510,feature_weight_511
0,557058,0.205733,0.317099,-0.241713,0.014264,-0.314002,-0.432338,0.152562,0.186511,0.140944,...,-0.034500,0.363631,0.183755,0.231815,0.042877,0.402666,0.016072,0.072968,-0.452475,-0.175632
1,155650,0.166665,-0.293123,-0.017896,-0.159223,0.246045,-0.138767,0.217365,-0.174787,-0.050760,...,0.276523,0.333738,0.277954,-0.124248,0.228173,0.029465,0.198626,-0.044919,0.661572,0.110016
2,425987,0.453440,0.174334,0.279131,0.144052,0.218536,0.110859,-0.090576,0.125406,0.110690,...,0.300525,-0.160008,0.052873,0.112233,-0.091779,-0.046790,0.280856,0.399340,0.305763,0.218547
3,262149,0.537410,0.165957,0.543808,-0.386137,-0.113595,0.046351,0.054061,0.247789,-0.502975,...,-0.113184,0.117278,0.011819,0.491052,-0.161192,0.868506,0.215711,0.108492,-0.607824,0.296478
4,557067,-0.193376,-0.058421,-0.074607,0.009510,-0.038007,0.011739,-0.141879,0.308456,0.392383,...,0.327086,0.413605,-0.092548,-0.091461,-0.025144,-0.202813,0.241059,0.014646,-0.073372,-0.242132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,368622,-0.330541,0.475290,0.010314,0.532330,-0.475708,0.443309,0.129722,-0.246515,0.317328,...,-0.000365,-0.001144,0.316298,0.075985,0.049807,-0.024745,0.322744,-0.502619,0.218220,-0.048334
2688,614382,0.275322,0.435353,0.091335,-0.033993,-0.504363,-0.133884,-0.367399,0.478917,0.033984,...,0.121871,-0.017587,0.141196,0.140291,0.354196,-0.044375,0.495297,0.369650,0.114305,0.281917
2689,557041,-0.598981,0.061932,-0.074026,0.171314,-0.250348,0.438974,-0.235911,0.036702,0.365656,...,0.039513,-0.141897,-0.232804,0.020774,0.542075,0.380731,0.189838,0.150242,-0.222899,-0.032958
2690,196596,0.024009,0.075810,0.314906,-0.302266,0.007095,0.024825,0.432788,-0.238054,0.210615,...,0.218091,0.377625,0.157004,0.265006,0.412147,0.088211,0.780171,0.079002,-0.009118,0.244956


In [34]:
# get a dataframe of feature weights for each obs. in X
X_train_fw = fw.merge(X_train_fw['pre_nucleus_id'], right_on ='pre_nucleus_id', left_on='nucleus_id')
X_val_fw = fw.merge(X_val['pre_nucleus_id'], right_on ='pre_nucleus_id', left_on='nucleus_id')
X_train_fw

,nucleus_id,feature_weight_0,feature_weight_1,feature_weight_2,feature_weight_3,feature_weight_4,feature_weight_5,feature_weight_6,feature_weight_7,feature_weight_8,...,feature_weight_503,feature_weight_504,feature_weight_505,feature_weight_506,feature_weight_507,feature_weight_508,feature_weight_509,feature_weight_510,feature_weight_511,pre_nucleus_id
0,557121,0.408289,0.051098,-0.510590,0.714646,-0.232921,-0.207754,-0.030060,0.369591,-0.367968,...,-0.123481,-0.214475,-0.599043,0.072844,0.522783,0.015744,0.359440,-0.021606,-0.146128,557121
1,557121,0.408289,0.051098,-0.510590,0.714646,-0.232921,-0.207754,-0.030060,0.369591,-0.367968,...,-0.123481,-0.214475,-0.599043,0.072844,0.522783,0.015744,0.359440,-0.021606,-0.146128,557121
2,557121,0.408289,0.051098,-0.510590,0.714646,-0.232921,-0.207754,-0.030060,0.369591,-0.367968,...,-0.123481,-0.214475,-0.599043,0.072844,0.522783,0.015744,0.359440,-0.021606,-0.146128,557121
3,557121,0.408289,0.051098,-0.510590,0.714646,-0.232921,-0.207754,-0.030060,0.369591,-0.367968,...,-0.123481,-0.214475,-0.599043,0.072844,0.522783,0.015744,0.359440,-0.021606,-0.146128,557121
4,557121,0.408289,0.051098,-0.510590,0.714646,-0.232921,-0.207754,-0.030060,0.369591,-0.367968,...,-0.123481,-0.214475,-0.599043,0.072844,0.522783,0.015744,0.359440,-0.021606,-0.146128,557121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239065,557030,-0.218660,-0.213865,-0.242872,0.169476,-0.038819,0.442058,0.012513,0.466563,0.399331,...,0.164836,-0.155090,0.051676,0.496228,0.104460,0.101426,-0.381078,-0.145225,0.364091,557030
239066,557030,-0.218660,-0.213865,-0.242872,0.169476,-0.038819,0.442058,0.012513,0.466563,0.399331,...,0.164836,-0.155090,0.051676,0.496228,0.104460,0.101426,-0.381078,-0.145225,0.364091,557030
239067,557030,-0.218660,-0.213865,-0.242872,0.169476,-0.038819,0.442058,0.012513,0.466563,0.399331,...,0.164836,-0.155090,0.051676,0.496228,0.104460,0.101426,-0.381078,-0.145225,0.364091,557030
239068,557030,-0.218660,-0.213865,-0.242872,0.169476,-0.038819,0.442058,0.012513,0.466563,0.399331,...,0.164836,-0.155090,0.051676,0.496228,0.104460,0.101426,-0.381078,-0.145225,0.364091,557030


In [153]:
fw_model = LogisticRegression(max_iter=500, solver='saga').fit(X_train_fw.drop(columns=['pre_nucleus_id', 'nucleus_id']).sort_index(axis=1), y_train)
preds = fw_model.predict(X_val_fw.drop(columns=['nucleus_id', 'pre_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val,preds)

0.5039385696827658

In [156]:
fw_model.coef_

array([[-1.87905109e-01, -1.59261052e-04, -4.11382870e-01,
        -9.04597872e-02,  7.74336663e-02, -1.84727554e-01,
        -6.31668802e-01, -8.41690600e-02, -7.06762806e-01,
         1.15974377e-01,  5.02110274e-01,  4.46436911e-01,
         1.09576016e-01, -5.96001979e-01, -1.19420203e-01,
         1.04446698e-01, -4.35287300e-01,  2.81188618e-01,
         1.23099389e-01,  3.02337789e-01,  4.94687974e-01,
         1.07056915e-01,  1.14215817e-01,  1.86713241e-01,
         2.54069941e-01, -4.38175894e-01, -3.00779466e-01,
         3.53740155e-01,  3.53836334e-02, -5.30516185e-01,
        -2.08428794e-01, -4.17765191e-01,  3.10133583e-01,
        -2.89535942e-01, -3.31267971e-01,  2.80790467e-01,
         1.89520012e-01, -4.58491034e-02,  5.98317267e-02,
         4.89907858e-01, -2.08101837e-01, -1.45131123e-01,
         7.31704104e-01, -4.66197202e-01,  2.66403867e-01,
         2.88544324e-01,  1.64289603e-01, -2.38119692e-01,
        -2.75648512e-01,  6.00507555e-01, -2.84215078e-0

In [157]:
imp_df = pd.DataFrame({"Features":X_val_fw.drop(columns=['pre_nucleus_id', 'nucleus_id']).columns, "Importance":fw_model.coef_[0]})
imp_df.sort_values('Importance', ascending=False)

,Features,Importance
506,feature_weight_506,1.049313
344,feature_weight_344,1.028434
102,feature_weight_102,0.965333
327,feature_weight_327,0.916856
176,feature_weight_176,0.885330
...,...,...
166,feature_weight_166,-0.799546
93,feature_weight_93,-0.847666
233,feature_weight_233,-1.067667
310,feature_weight_310,-1.143740


In [83]:
# Include FW 101 in the training data
def add_feature(df, fw, feature_name):
    """"
    Adds a feature weight to the dataframe
    """
    df = df.merge(fw[[feature_name, 'nucleus_id']], left_on='pre_nucleus_id', right_on='nucleus_id')
    df = df.drop(columns='nucleus_id')
    return df

In [84]:
X_train_feat = add_feature(X_train_feat, fw, 'feature_weight_398')
X_val_feat = add_feature(X_val_feat, fw, 'feature_weight_398')
X_query_feat = add_feature(X_query_feat, fw, 'feature_weight_398')

In [85]:
X_train_feat

,ID,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,pre_oracle,pre_test_score,post_oracle,post_test_score,compartment,pre_brain_area,...,post_morph_embeddings,me_similarity,fw_similarity,axonal_coords,dendritic_coords,pre_rf_coords,post_rf_coords,pre_nucleus_coords,nuclei_adp_dist,feature_weight_398
0,42593,-1.515697,2.007164,0.931055,0.037561,0.787980,1.191761,-0.205420,axon,RL,...,"[1.0723994970321655, -0.7540942430496216, 0.11...",-0.934986,-0.182150,"[1187660, 411978, 1089020]","[1187390, 412220, 1089160]","[882.7244675159454, 519.1477453708649]","[858.5207009315491, 608.4342455863953]","[1283392, 621504, 1020280]",0.076496,0.021513
1,42594,-1.225578,1.070301,0.268030,0.037561,0.787980,1.191761,-0.205420,axon,RL,...,"[1.0723994970321655, -0.7540942430496216, 0.11...",-0.934986,-0.182150,"[1204580, 682542, 873138]","[1204640, 682870, 873890]","[882.7244675159454, 519.1477453708649]","[858.5207009315491, 608.4342455863953]","[1283392, 621504, 1020280]",-0.306546,0.021513
2,42595,0.632301,2.099967,0.905442,0.037561,0.787980,1.191761,-0.205420,axon,RL,...,"[1.0723994970321655, -0.7540942430496216, 0.11...",-0.934986,-0.182150,"[1191790, 403683, 1093180]","[1188590, 402414, 1092660]","[882.7244675159454, 519.1477453708649]","[858.5207009315491, 608.4342455863953]","[1283392, 621504, 1020280]",0.118612,0.021513
3,42596,0.645635,1.931656,0.956222,0.037561,0.787980,1.191761,-0.205420,axon,RL,...,"[1.0723994970321655, -0.7540942430496216, 0.11...",-0.934986,-0.182150,"[1184320, 419286, 1082930]","[1186620, 419721, 1085540]","[882.7244675159454, 519.1477453708649]","[858.5207009315491, 608.4342455863953]","[1283392, 621504, 1020280]",0.035778,0.021513
4,42597,1.334344,1.668055,-0.042977,0.037561,0.787980,1.191761,-0.205420,axon,RL,...,"[1.0723994970321655, -0.7540942430496216, 0.11...",-0.934986,-0.182150,"[1189150, 673302, 944202]","[1188790, 677771, 942901]","[882.7244675159454, 519.1477453708649]","[858.5207009315491, 608.4342455863953]","[1283392, 621504, 1020280]",-0.587712,0.021513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239065,226206,-0.581809,-0.728975,0.850357,0.221486,0.550581,0.408128,-1.431450,apical,RL,...,"[-0.8487598896026611, -0.4065110981464386, -0....",-0.984540,0.015884,"[1242320, 386568, 835506]","[1242730, 384648, 835346]","[823.984922170639, 584.9846363067627]","[948.8567638397216, 562.5469064712524]","[1267840, 625088, 983200]",0.328965,-0.442345
239066,227082,-1.233690,-0.560997,-0.682696,0.221486,0.550581,-0.277208,-0.747283,basal,RL,...,"[0.7182760238647461, 0.2712369859218597, -0.43...",0.653998,2.662264,"[1307560, 720111, 821478]","[1307660, 721056, 822030]","[823.984922170639, 584.9846363067627]","[876.9663333892822, 475.53187251091]","[1267840, 625088, 983200]",-0.220960,-0.442345
239067,227102,-1.111118,-0.325588,0.167203,0.221486,0.550581,-1.200803,-1.403504,basal,RL,...,"[0.2488172650337219, -0.0253329630941152, -0.6...",0.660586,-1.209141,"[1253340, 647535, 913773]","[1254210, 648092, 913497]","[823.984922170639, 584.9846363067627]","[840.3159034252167, 610.6111478805542]","[1267840, 625088, 983200]",-0.938165,-0.442345
239068,226434,-0.162126,-0.180224,0.585322,0.221486,0.550581,0.765834,1.948422,basal,RL,...,"[1.3395723104476929, -0.0049529541283845, -0.4...",0.827080,-0.676461,"[1350760, 712929, 871647]","[1350490, 714924, 870280]","[823.984922170639, 584.9846363067627]","[870.390248298645, 547.6739072799683]","[1267840, 625088, 983200]",-0.387860,-0.442345


In [73]:
fw = pd.read_csv(feature_path)
fw = fw.drop(columns='nucleus_id')
np.argmax(fw.std(axis=0))

398

# One-Hot Encode All Categorical Variables

In [5]:
def one_hot(column, df, suffix=''):
    """
    one-hot encodes this shit
    """
    cats = pd.unique(df[column])

    for cat in cats:
        new_col = cat+suffix
        df[new_col] = df[column]==cat
        df[new_col] = df[new_col].astype('int')
    
    df = df.drop(columns=column)
    return df

In [6]:
# one-hot encode brain areas for all
X_train_feat = one_hot('pre_brain_area', X_train_feat, '_pre')
X_train_feat = one_hot('post_brain_area', X_train_feat, '_post')

X_val_feat = one_hot('pre_brain_area', X_val_feat, '_pre')
X_val_feat = one_hot('post_brain_area', X_val_feat, '_post')

X_query_feat = one_hot('pre_brain_area', X_query_feat, '_pre')
X_query_feat = one_hot('post_brain_area', X_query_feat, '_post')

In [7]:
# encode brain areas
area1 = ["basal", "soma"]
area2 = ["axon", "apical", "oblique", "apical_shaft"]
area3 = ["apical_tuft"]

def area_cols(df):
    df["area1"] = df["compartment"].isin(area1).astype('int')
    df["area2"] = df["compartment"].isin(area2).astype('int')
    df["area3"] = df["compartment"].isin(area3).astype('int')
    df.drop(columns='compartment')
    return df

In [8]:
X_train_feat = area_cols(X_train_feat)
X_val_feat = area_cols(X_val_feat)
X_query_feat = area_cols(X_query_feat)

In [9]:
X_train_feat = X_train_feat.select_dtypes('number')
X_val_feat = X_val_feat.select_dtypes('number')
X_query_feat = X_query_feat.select_dtypes('number')

In [10]:
X_train_feat.dtypes

ID                                  int64
adp_dist                          float64
post_skeletal_distance_to_soma    float64
pre_skeletal_distance_to_soma     float64
pre_oracle                        float64
pre_test_score                    float64
post_oracle                       float64
post_test_score                   float64
post_nucleus_x                    float64
post_nucleus_y                    float64
post_nucleus_z                    float64
pre_nucleus_id                      int64
post_nucleus_id                     int64
me_similarity                     float64
fw_similarity                     float64
nuclei_adp_dist                   float64
AL_pre                              int64
RL_pre                              int64
V1_pre                              int64
V1_post                             int64
AL_post                             int64
RL_post                             int64
area1                               int64
area2                             

# Logistic Regression with Lasso For Feature Selection

In [11]:
fw_model = LogisticRegression(max_iter=500, solver='saga').fit(X_train_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1), y_train)


In [12]:
preds = fw_model.predict(X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val,preds)

0.7458810383129851

In [13]:
log_feats = pd.DataFrame({"features":X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).columns})
log_feats['coefs']=abs(fw_model.coef_[0])
log_feats.sort_values(by='coefs', ascending=False, inplace=True)

In [44]:
known_3 = ['adp_dist', 'post_skeletal_distance_to_soma', 'pre_skeletal_distance_to_soma']

In [37]:
log_feats['features'][0:5]

6     post_test_score
8      post_nucleus_y
9      post_nucleus_z
17            V1_post
4      pre_test_score
Name: features, dtype: object

In [38]:
log_feats["features"][0:5][~log_feats['features'][0:5].isin(known_3)]

6     post_test_score
8      post_nucleus_y
9      post_nucleus_z
17            V1_post
4      pre_test_score
Name: features, dtype: object

In [75]:
full_set = np.append(known_3, log_feats["features"][0:6][~log_feats['features'][0:6].isin(known_3)])
full_set = list(full_set)
full_set.append('fw_similarity')
# full_set.append('me_similarity')
# full_set = known_3
# full_set.append('V1_post')
full_set

['adp_dist',
 'post_skeletal_distance_to_soma',
 'pre_skeletal_distance_to_soma',
 'post_test_score',
 'post_nucleus_y',
 'post_nucleus_z',
 'V1_post',
 'pre_test_score',
 'fw_similarity']

In [46]:
fw_model = LogisticRegression(max_iter=500, solver='saga').fit(X_train_feat[full_set].sort_index(axis=1), y_train)
preds = fw_model.predict(X_query_feat[full_set].sort_index(axis=1))
balanced_accuracy_score(y_query,preds)

0.7268430863746624

# XGBoost for Feature Selection

In [236]:
xg_model = XGBClassifier(n_estimators=1, grow_policy='lossguide', max_leaves=0)
xg_model.fit(X_train_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1), y_train)
preds = xg_model.predict(X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val, preds)

0.7017671975408951

# Bagging for Feature Selection

In [238]:
bag_model = BaggingClassifier()
bag_model.fit(X_train_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1), y_train)
preds = bag_model.predict(X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val, preds)

0.5018827521747283

# Select Top Fucking Features

In [118]:
def feature_selector(df, features, top):
    """ 
    Function that selects the best features

    Inputs:
        - df: dataframe 
        - features: numpy array of features ranked by their importance as determined by different methods
        - top: (int) number of top features to consider
    """
    use_df = df.copy()
    use_df = use_df[np.array(features["features"][0:top])]

    return use_df

In [39]:
X_train_use = feature_selector(X_train_feat, log_feats, 5)
X_val_use = feature_selector(X_val_feat, log_feats, 5)
X_query_use = feature_selector(X_query_feat, log_feats, 5)

In [37]:
X_query_use

,post_test_score,post_nucleus_y,V1_post,post_nucleus_z,pre_test_score
25710,-0.205420,-1.113624,0,0.557703,-0.980009
25711,-0.205420,-1.113624,0,0.557703,-0.980009
25712,-0.205420,-1.113624,0,0.557703,-0.980009
25713,-0.108867,-1.178216,0,-1.581511,-0.980009
25714,-0.108867,-1.178216,0,-1.581511,-0.980009
...,...,...,...,...,...
166059,-1.077152,-1.391026,0,0.466812,0.741950
166060,-1.077152,-1.391026,0,0.466812,0.741950
166061,-1.077152,-1.391026,0,0.466812,0.741950
166062,-1.077152,-1.391026,0,0.466812,0.741950


# Make Predictions idk

In [38]:
# run if NOT feature selection
X_train_use = X_train_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1)
X_val_use = X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1)
X_query_use = X_query_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1)

In [47]:
# try this one out
X_train_use =  X_train_feat[full_set]
X_val_use =  X_val_feat[full_set]
X_query_use =  X_query_feat[full_set]

In [98]:
# models = {"RFC": RandomForestClassifier(), "LDA": LinearDiscriminantAnalysis()}
# param_grids = {
#             "RFC": [{'n_estimators' : 1000, 'n_jobs' : -1}, 
#                     {'n_estimators' : 10000, 'n_jobs' : -1}],
#             "LDA": [{'solver' : 'lsqr'},
#                     {'solver' : 'eigen'}]
#             }

# OPTION 2 FOR LOGREG PERFORMED BEST ON FULL DATASET
models = {"XGBoost": XGBClassifier(), 'LogReg':LogisticRegression()}
param_grids = {
            "XGBoost": [{'booster' : 'gbtree', 'eta':0.01, 'max_depth':10}, 
                    {'booster' : 'gblinear', 'eta':0.01, 'max_depth':10},
                    {'booster' : 'gblinear', 'eta':0.05, 'max_depth':15}],
        # "LogReg": [{'max_iter':500, 'penalty':'l2'},
        #            {'max_iter':500, 'penalty':'l1', 'solver':'liblinear'},
        #            {'max_iter':500, 'penalty':'elasticnet', 'l1_ratio':0.1, 'solver':'saga'},
        #            {'max_iter':500, 'penalty':'elasticnet', 'l1_ratio':0.2, 'solver':'saga'},
        #            {'max_iter':500, 'penalty':'elasticnet', 'l1_ratio':0.3, 'solver':'saga'},
        #            {'max_iter':500, 'penalty':'elasticnet', 'l1_ratio':0.4, 'solver':'saga'},
        #            {'max_iter':500, 'penalty':'elasticnet', 'l1_ratio':0.5, 'solver':'saga'}]
            }

# BEST: {'max_iter':500, 'penalty':'l2', 'solver':'newton-cholesky'} used on the top 5 features
# models = {'LogReg':LogisticRegression()}
# param_grids = {
#         "LogReg": [{'max_iter':500, 'penalty':'l2', 'solver':'saga'},
#                    {'max_iter':500, 'penalty':'l2', 'solver':'newton-cholesky'},
#                    {'max_iter':500, 'penalty':'l1', 'solver':'saga'},
#                    {'max_iter':500, 'penalty':'elasticnet', 'l1_ratio':0.6, 'solver':'saga'}]
#             }

In [99]:
optimum_models = dict()
accuracies = dict()
for model in param_grids:
    classifier = models[model]
    prev_acc = 0
    optimum_param = dict()
    accuracy = []
    for values in param_grids[model]:
        
        #Fitting to the training data with selected hyperparameters 0.732002483867241
        classifier.set_params(**values)
        print(classifier)
        classifier.fit(X_train_use, y_train)

        #Finding the balanced accuracy
        y_hat = classifier.predict(X_query_use)
        print(y_hat)
        print(y_query)
        balanced_accuracy = balanced_accuracy_score(y_query, y_hat)
        accuracy.append(balanced_accuracy)
        print(accuracy)
        if balanced_accuracy > prev_acc:
            print(prev_acc, balanced_accuracy)
            prev_acc = balanced_accuracy
            optimum_param = values
            
        
    accuracies[model] = accuracy
    optimum_models[model] = classifier.set_params(**optimum_param)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.01, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)
[0 0 0 ... 0 0 0]
0         False
1         False
2         False
3         False
4         False
          ...  
179187    False
179188    False
179189    False
179190    False
179191    False
Name: connected, Length: 32518, dtype: bool
[0.66620137

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:11:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)


[1 1 0 ... 0 0 0]
0         False
1         False
2         False
3         False
4         False
          ...  
179187    False
179188    False
179189    False
179190    False
179191    False
Name: connected, Length: 32518, dtype: bool
[0.6662013772566536, 0.7325298791988968]
0.6662013772566536 0.7325298791988968
XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.05, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_est

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:11:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)


[1 1 0 ... 0 0 0]
0         False
1         False
2         False
3         False
4         False
          ...  
179187    False
179188    False
179189    False
179190    False
179191    False
Name: connected, Length: 32518, dtype: bool
[0.6662013772566536, 0.7325298791988968, 0.7234400599177702]


In [100]:
accuracies

{'XGBoost': [0.6662013772566536, 0.7325298791988968, 0.7234400599177702]}

In [101]:
optimum_models['XGBoost']

XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.01, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

# Make A Submission

In [102]:
# submission comment
print('Features used:', X_train_use.columns, '\nOptimum Model Parameters:', optimum_models['XGBoost'])

Features used: Index(['adp_dist', 'post_skeletal_distance_to_soma',
       'pre_skeletal_distance_to_soma', 'post_test_score', 'post_nucleus_y',
       'post_nucleus_z', 'V1_post', 'pre_test_score', 'fw_similarity'],
      dtype='object') 
Optimum Model Parameters: XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.01, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parall

In [109]:
leaderboard_path = "../Data/leaderboard_data.csv"
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)
sub_data = area_cols(sub_data)

In [110]:
sub_data.columns

Index(['ID', 'adp_dist', 'post_skeletal_distance_to_soma',
       'pre_skeletal_distance_to_soma', 'pre_oracle', 'pre_test_score',
       'post_oracle', 'post_test_score', 'compartment', 'pre_brain_area',
       'post_brain_area', 'post_nucleus_x', 'post_nucleus_y', 'post_nucleus_z',
       'pre_nucleus_id', 'post_nucleus_id', 'pre_feature_weights',
       'post_feature_weights', 'pre_morph_embeddings', 'post_morph_embeddings',
       'me_similarity', 'fw_similarity', 'axonal_coords', 'dendritic_coords',
       'pre_rf_coords', 'post_rf_coords', 'pre_nucleus_coords',
       'nuclei_adp_dist', 'area1', 'area2', 'area3'],
      dtype='object')

In [111]:
sub_data = one_hot('pre_brain_area', sub_data, '_pre')
sub_data = one_hot('post_brain_area', sub_data, '_post')
# sub_data = one_hot('projection_group', sub_data)

In [114]:
full_set = np.append(known_3, log_feats["features"][0:6][~log_feats['features'][0:6].isin(known_3)])
full_set = list(full_set)
full_set.append('fw_similarity')

In [116]:
sub_set = full_set
sub_set.append('ID')
sub_data_use = sub_data[full_set]
full_set

['adp_dist',
 'post_skeletal_distance_to_soma',
 'pre_skeletal_distance_to_soma',
 'post_test_score',
 'post_nucleus_y',
 'post_nucleus_z',
 'V1_post',
 'pre_test_score',
 'fw_similarity',
 'ID']

In [117]:
sub_data_use["connected"]=optimum_models['XGBoost'].predict(sub_data_use.drop(columns='ID'))

/var/folders/b8/zl2jm8vs42b9q_gbk09_pdn40000gn/T/ipykernel_45666/2008015115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data_use["connected"]=optimum_models['XGBoost'].predict(sub_data_use.drop(columns='ID'))


In [118]:
submission_data = sub_data_use.filter(['ID','connected'])
submission_data['connected'] = submission_data['connected']==1

In [119]:
submission_data.to_csv('submission_data.csv',index=False)

In [120]:
preds = optimum_models['XGBoost'].predict(X_val_use)==1
balanced_accuracy_score(y_val, preds)

0.7443735103690882